In [ ]:
!pip install transformers

In [ ]:
import torch
torch.__version__

'2.1.0+cu121'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("neulab/omnitab-large-finetuned-wtq")
model = AutoModelForSeq2SeqLM.from_pretrained("neulab/omnitab-large-finetuned-wtq")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/979 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/813M [00:00<?, ?B/s]

In [ ]:
import pandas as pd

In [ ]:
data = {"Actors": ["", "Leonardo Di Caprio", "George Clooney"], "Number of movies": ["87", "53", "69"]}
table = pd.DataFrame.from_dict(data)
questions = "how many movies does Leonardo Di Caprio have?"
encoding = tokenizer(table, questions, padding=True, return_tensors="pt")
print(encoding)
print(len(encoding['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
for key, value in encoding.items():
    print('{}: {}'.format(key, value))

{'input_ids': tensor([[    0,  9178,   171,  4133,   473, 20683,  3643,  5133, 12834,    33,
           116, 11311,  4832,  1783,   994,  1721, 12270,     9,  4133,  3236,
           112,  4832,  4770,  9386,  1721,  8176,  3236,   132,  4832, 20683,
          3643,  5133, 12834,  1721,  4268,  3236,   155,  4832,  1655, 21627,
          6071,  1721,  5913,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
44
['<s>', 'how', 'Ġmany', 'Ġmovies', 'Ġdoes', 'ĠLeonardo', 'ĠDi', 'ĠCap', 'rio', 'Ġhave', '?', 'Ġcol', 'Ġ:', 'ĠAct', 'ors', 'Ġ|', 'ĠNumber', 'Ġof', 'Ġmovies', 'Ġrow', 'Ġ1', 'Ġ:', 'ĠBrad', 'ĠPitt', 'Ġ|', 'Ġ87', 'Ġrow', 'Ġ2', 'Ġ:', 'ĠLeonardo', 'ĠDi', 'ĠCap', 'rio', 'Ġ|', 'Ġ53', 'Ġrow', 'Ġ3', 'Ġ:', 'ĠGeorge', 'ĠClo', 'oney', 'Ġ|', 'Ġ69', '</s>']
input_ids: tensor([[    0,  9178,   171,  4133,   473, 20683,  3643,  5133, 12834,    33,
           116, 11311,

In [ ]:
tokenizer.model_max_length

1024

In [ ]:
import pickle
import pandas as pd

In [ ]:
with open('train_tables.pkl', 'rb') as f:
  x = pickle.load(f)
with open('dev_tables_new.pkl', 'rb') as f:
  y = pickle.load(f)
with open('question_answer_pairs_new.pkl', 'rb') as f:
  z = pickle.load(f)

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#qlist = []
#Table only
c = 0
c2 = 0
s = 0
f1 = 0
for i in range(50):
  qlist = []
  gold_list = []
  for j in range(len(z[i])):
    qlist.append(z[i][j]['ques'])
    gold_list.append(z[i][j]['ans'])

  encoding = tokenizer(y[i], qlist, padding=True, return_tensors="pt")
  print(len(encoding['input_ids'][0]))
  s += len(encoding['input_ids'][0])
  if(len(encoding['input_ids'][0])>512):
    c2+=1
  if(len(encoding['input_ids'][0])<=512):
    outputs = model.generate(**encoding)
    x = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    print(x)
    print(gold_list)

    val = 0
    for p in range(len(x)):
      y1 = word_tokenize(x[p])
      z1 = word_tokenize(gold_list[p])
      #gh = compute_f1(y1, z1)
      #c3 += gh
      #ij = compute_exact(y1, z1)
      #c4 += ij
      ct = 0
      for j in range(len(y1)):
        for k in range(len(z1)):
          if(y1[j].lower() == z1[k].lower() or y1[j].lower() == " " + z1[k].lower() or z1[k].lower() == " " + y1[j].lower()):
            ct+=1

      if(ct!=0):
        prec = 1.0*ct/(len(y1))
        rec = 1.0*ct/(len(z1))
        val += (2*prec*rec)/(prec+rec)

    val = val/len(x)
    f1 += val
    print(f1)
    #print()

    for j in range(len(z[i])):
      if(x[j].lower() == " " + gold_list[j].lower() or x[j].lower() == " " + gold_list[j].lower() + "0" or x[j].lower() == " [bold] " + gold_list[j].lower()):
        c+=1
    print(c)
    print("----------")
    print(i)
    print("***************")
print(c)
print("Exact match:")
print(c/199)
print(c2)
print(s)
#print(f1)

195
[' 52.2', ' 5.5', ' 20.2', ' Yes']
['52.2', '61.6', '52.6', 'less than']
0.25
1
----------
0
***************
100
[' 22.7', ' 129.7', ' 7.6, 22.7', ' 188.2']
['22.7', '129.7', 'Dataset-Moderate, Value-27.3', '137.3']
0.8333333333333334
3
----------
1
***************
212
[' 72.83', ' 1.34', ' UD v1.3', ' CoNLL08']
['73.34', '2.22', 'UD v1.3', 'CoNLL08']
1.3333333333333335
5
----------
2
***************
181
[' TOPIC', ' PART_WHOLE', ' TOPIC', ' +27.23']
['USAGE', 'COMPARE', 'TOPIC', '+27.23']
1.8333333333333335
7
----------
3
***************
309
[' 66.84', ' 28.30', ' 67.71', ' 34.35']
['66.84', '30.22', '67.84', '34.35']
2.3333333333333335
9
----------
4
***************
649
76
[' LSTM-Parser', ' 64.74', ' 9.40±13.57', ' 64.74']
['Taggers', '±13.57', '9.40±13.57', '64.74±1.97']
2.5833333333333335
10
----------
6
***************
849
181
[' 11.42', ' 4,558', ' 17.69', ' Dev']
['11.42', '3500', '17.69', 'TEST']
3.0833333333333335
12
----------
8
***************
834
85
[' 14', ' Low', ' 9

In [ ]:
print(c/179)

0.39664804469273746


In [ ]:
print(f1/45)

0.4672408293460925


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk import word_tokenize

In [ ]:
import string

In [ ]:
from collections import Counter

In [ ]:
dfx = pd.read_json('test.json')
dfx = dfx.T
dfx

,paper,paper_id,table_caption,table_column_names,table_content_values,text
0,Improving the Expressiveness of Deep Learning ...,1809.00832v1,Table 2: Throughput for processing the TreeLST...,"[Batch size, Throughput (instances/s) Inferenc...","[[Batch size, Iter, Recur, Fold, Iter, Recur, ...",Table 2 compares the throughput of performing ...
1,Improving the Expressiveness of Deep Learning ...,1809.00832v1,Table 1: Throughput for the TreeRNN model impl...,"[Batch size, Throughput (instances/s) Balanced...","[[1, 46.7, 27.3, 7.6], [10, 125.2, 78.2, 22.7]...",Table 1 shows the throughput of training the T...
2,Syntactic Dependency Representations in Neural...,1805.11461v1,Table 2: Hyper parameter optimization results ...,"[[BOLD] Representation, [BOLD] Hyper parameter...","[[CoNLL08, 4-5, 1000, Softplus, 1.15e+01, 1.13...",Table 2 presents the optimal values for each c...
3,Syntactic Dependency Representations in Neural...,1805.11461v1,Table 1: Effect of using the shortest dependen...,"[[BOLD] Relation, [BOLD] best F1 (in 5-fold) w...","[[USAGE, 60.34, 80.24, + 19.90], [MODEL-FEATUR...",We find that the effect of syntactic structure...
4,Neural End-to-End Learning for Computational A...,1704.06104v2,Table 3: Performance of MTL sequence tagging a...,"[[EMPTY], C-F1 100%, C-F1 50%, R-F1 100%, R-F1...","[[Y-3, 49.59, 65.37, 26.28, 37.00, 34.35, 47.2...","Accordingly, as Table 3 shows for the essay le..."
...,...,...,...,...,...,...
95,On the difficulty of a distributional semantic...,1803.08869v2,Table 1: Results on Synthetically Spoken COCO....,"[[EMPTY], Recall@10 (%), Median rank, RSAimage]","[[VGS, 27, 6, 0.4], [SegMatch, [BOLD] 10, [BOL...",Table 1 shows the evaluation results on synthe...
96,What do Deep Networks Like to Read?,1909.04547v1,Table 1: Example sentences of the different cl...,"[Orig, <u> turns in a <u> screenplay that <u> ...","[[DAN, <u> turns in a <u> screenplay screenpla...",Using SIFT we observed substantial differences...
97,What do Deep Networks Like to Read?,1909.04547v1,Table 2: Part-of-Speech (POS) changes in SST-2...,"[[EMPTY], <bold>RNN</bold>, <bold>CNN</bold>, ...","[[Nouns, +63, −3, +<bold>93</bold>], [DT, −29,...","In order to identify differences, we conducted..."
98,What do Deep Networks Like to Read?,1909.04547v1,Table 3: Sentiment score changes in SST-2. The...,"[[EMPTY], <bold>RNN</bold>, <bold>CNN</bold>, ...","[[Positive, +9.7, +4.3, +<bold>23.6</bold>], [...",In order to automatically identify if SIFT ret...


In [ ]:
val = []
for i in range(100):
  val.append(dfx['table_caption'][i])
val2 = []
for i in range(100):
  val2.append(dfx['text'][i])


In [ ]:
for i in range(len(val)):
  val[i] = val[i].replace('\xa0','')

for i in range(len(val2)):
  val2[i] = val2[i].replace(' [CONTINUE] ','')

In [ ]:
import copy
a = copy.copy(z)

In [ ]:
for i in range(len(a)):
  for j in range(len(a[i])):
    a[i][j]['ques'] = a[i][j]['ques'] + " " + str(val[i])

In [ ]:
#qlist = []
#Table + caption
c = 0
c2 = 0
len_to_truncate = []
for i in range(50):
  qlist = []
  gold_list = []
  for j in range(len(a[i])):
    qlist.append(a[i][j]['ques'])
    gold_list.append(a[i][j]['ans'])

  encoding = tokenizer(y[i], qlist, padding=True, truncation=True, return_tensors="pt")
  print(len(encoding['input_ids'][0]))
  print(tokenizer.model_max_length - len(encoding['input_ids'][0]))
  len_to_truncate.append(tokenizer.model_max_length - len(encoding['input_ids'][0]))
  if(len(encoding['input_ids'][0])>512):
    c2+=1
  outputs = model.generate(**encoding)

  x = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  print(x)
  print(gold_list)

  for j in range(len(a[i])):
    if(x[j].lower() == " " + gold_list[j].lower() or x[j].lower() == " " + gold_list[j].lower() + "0" or x[j].lower() == " [bold] " + gold_list[j].lower()):
      c+=1
  print(c)
  print("----------")
  print(i)
  print("***************")
print(c)
print("Exact match:")
print(c/199)
print(c2)

353
671
[' 37.5', ' 9.0', ' 24.93', ' Yes']
['52.2', '61.6', '52.6', 'less than']
0
----------
0
***************
319
705
[' 22.7', ' 97.9', ' Balanced dataset, 7.6', ' 48.6']
['22.7', '129.7', 'Dataset-Moderate, Value-27.3', '137.3']
1
----------
1
***************
327
697
[' 72.83', ' 5.34', ' UD v1.3', ' CoNLL08']
['73.34', '2.22', 'UD v1.3', 'CoNLL08']
3
----------
2
***************
232
792
[' Part_WHOLE', ' COMPARE', ' TOPIC', ' +27.23']
['USAGE', 'COMPARE', 'TOPIC', '+27.23']
6
----------
3
***************
427
597
[' 47.92', ' 28.30', ' 47.25', ' 37.35']
['66.84', '30.22', '67.84', '34.35']
6
----------
4
***************
873
151
[' 0', ' 33.72', ' 54.17', ' 73.02']
['6.90', '29.56', '77.19', '50.51']
6
----------
5
***************
165
859
[' taggers', ' Paragraph', ' STagBLCC', ' LSTM-Parser']
['Taggers', '±13.57', '9.40±13.57', '64.74±1.97']
7
----------
6
***************
1024
0
[' 39.23', ' 10.31', ' 37.94', ' 0.5036']
['36.85', '35.03', '37.94', '0.3855']
8
----------
7
********

In [ ]:
x[184]

In [ ]:
import copy
b = copy.copy(a)

In [ ]:
for i in range(len(b)):
  for j in range(len(b[i])):
    b[i][j]['ques'] = b[i][j]['ques'] + " " + str(val2[i])

In [ ]:
print(a[1][2]['ques'])
print(b[1][2]['ques'])

In [ ]:
#qlist = []
#Table + caption + mention
c = 0
c2 = 0
f1 = 0
len_to_truncate = []
for i in range(50):
  qlist = []
  gold_list = []
  for j in range(len(b[i])):
    qlist.append(b[i][j]['ques'])
    gold_list.append(b[i][j]['ans'])

  encoding = tokenizer(y[i], qlist, padding=True, truncation=True, return_tensors="pt")
  print(len(encoding['input_ids'][0]))
  print(tokenizer.model_max_length - len(encoding['input_ids'][0]))
  len_to_truncate.append(tokenizer.model_max_length - len(encoding['input_ids'][0]))
  if(len(encoding['input_ids'][0])>=1024):
    print(i)
    c2+=1
  outputs = model.generate(**encoding)

  x = tokenizer.batch_decode(outputs, skip_special_tokens=True)
  print(x)
  print(gold_list)

  val = 0
  for p in range(len(x)):
    y1 = word_tokenize(x[p])
    z1 = word_tokenize(gold_list[p])
    #gh = compute_f1(y1, z1)
    #c3 += gh
    #ij = compute_exact(y1, z1)
    #c4 += ij
    ct = 0
    for j in range(len(y1)):
      for k in range(len(z1)):
        if(y1[j].lower() == z1[k].lower() or y1[j].lower() == " " + z1[k].lower() or z1[k].lower() == " " + y1[j].lower()):
          ct+=1

    if(ct!=0):
      prec = 1.0*ct/(len(y1))
      rec = 1.0*ct/(len(z1))
      val += (2*prec*rec)/(prec+rec)

  val = val/len(x)
  f1 += val
  print(f1)

  for j in range(len(b[i])):
    if(x[j].lower() == " " + gold_list[j].lower() or x[j].lower() == " " + gold_list[j].lower() + "0" or x[j].lower() == " [bold] " + gold_list[j].lower()):
      c+=1
  print(c)
  print("----------")
  print(i)
  print("***************")
print(c)
print("Exact match:")
print(c/199)
print(c2)
print(f1/50)

353
671
[' 37.5', ' 9.0', ' 24.93', ' Yes']
['52.2', '61.6', '52.6', 'less than']
0.0
0
----------
0
***************
319
705
[' 22.7', ' 97.9', ' Balanced dataset, 7.6', ' 48.6']
['22.7', '129.7', 'Dataset-Moderate, Value-27.3', '137.3']
0.32142857142857145
1
----------
1
***************
327
697
[' 72.83', ' 5.34', ' UD v1.3', ' CoNLL08']
['73.34', '2.22', 'UD v1.3', 'CoNLL08']
0.8214285714285714
3
----------
2
***************
232
792
[' Part_WHOLE', ' COMPARE', ' TOPIC', ' +27.23']
['USAGE', 'COMPARE', 'TOPIC', '+27.23']
1.5714285714285714
6
----------
3
***************
427
597
[' 47.92', ' 28.30', ' 47.25', ' 37.35']
['66.84', '30.22', '67.84', '34.35']
1.5714285714285714
6
----------
4
***************
873
151
[' 0', ' 33.72', ' 54.17', ' 73.02']
['6.90', '29.56', '77.19', '50.51']
1.5714285714285714
6
----------
5
***************
165
859
[' taggers', ' Paragraph', ' STagBLCC', ' LSTM-Parser']
['Taggers', '±13.57', '9.40±13.57', '64.74±1.97']
1.8214285714285714
7
----------
6
*******

In [ ]:
def

In [ ]:
encoding_for_text = tokenizer(y[7], b[9][2]['ques'][:946], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[9][1]['ques'][:940], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[9][0]['ques'][:941], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[9][3]['ques'][:942], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[20][3]['ques'][:920], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[20][2]['ques'][:920], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[20][1]['ques'][:929], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
encoding_for_text = tokenizer(y[7], b[20][0]['ques'][:911], padding=True, return_tensors="pt")
print(encoding_for_text['input_ids'][0])
print(len(encoding_for_text['input_ids'][0]))
print(tokenizer.convert_ids_to_tokens(encoding_for_text['input_ids'][0]))


In [ ]:
#Truncation
b[7][0]['ques'] = b[7][0]['ques'][:998]
b[7][1]['ques'] = b[7][1]['ques'][:1002]
b[7][2]['ques'] = b[7][2]['ques'][:1005]
b[7][3]['ques'] = b[7][3]['ques'][:1002]

b[9][0]['ques'] = b[9][0]['ques'][:941]
b[9][1]['ques'] = b[9][1]['ques'][:940]
b[9][2]['ques'] = b[9][2]['ques'][:946]
b[9][3]['ques'] = b[9][3]['ques'][:942]

b[20][0]['ques'] = b[20][0]['ques'][:911]
b[20][1]['ques'] = b[20][1]['ques'][:929]
b[20][2]['ques'] = b[20][2]['ques'][:920]
b[20][3]['ques'] = b[20][3]['ques'][:920]



In [ ]:
print(b[20][0]['ques'][:911])